<a href="https://colab.research.google.com/github/Tvorozh0k/ssu-ml-course/blob/main/4_knn_implementation/ML_Penguins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подключение библиотек

In [1]:
#@title Подключение и настройка библиотеки Pandas

import pandas as pd

from google.colab import drive
from google.colab import data_table
from google.colab.data_table import DataTable

DataTable.num_rows_per_page = 25
data_table.enable_dataframe_formatter()

In [2]:
#@title Подключение графических библиотек

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
#@title Подключение математических библиотек

import random
import numpy as np
from scipy.spatial import distance, Voronoi

In [4]:
#@title Подключение библиотеки Sklearn

# модели машинного обучения
from sklearn.neighbors import KNeighborsClassifier

# оценки качества моделей
from sklearn.metrics import classification_report

# преобразование признаков
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder

# понижение размерности
from sklearn.neighbors import NeighborhoodComponentsAnalysis

# конструкторы
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Загрузка датасета

In [5]:
#@title Датасет

df = pd.read_csv('https://raw.githubusercontent.com/Tvorozh0k/ssu-ml-course/refs/heads/main/4_knn_implementation/data/penguins.csv')
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male


In [6]:
#@title Информация о признаках

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [7]:
#@title Пропуски

df.isnull().sum()

,0
species,0
island,0
bill_length_mm,2
bill_depth_mm,2
flipper_length_mm,2
body_mass_g,2
sex,11


## Признаки

In [ ]:
#@title Корреляционная матрица

fig = px.imshow(df.corr(), width=500, height=500, color_continuous_scale=['#f84f31', '#ffffff', '#23c552'], text_auto=True)
fig.show()

<ipython-input-22-6f79d350eff0>:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



### Species - категориальный признак (виды пингвинов)

In [ ]:
#@title Информация

df['species'].describe()

count        344
unique         3
top       Adelie
freq         152
Name: species, dtype: object

In [ ]:
#@title Значения

df['species'].value_counts()

Adelie       152
Gentoo       124
Chinstrap     68
Name: species, dtype: int64

In [ ]:
#@title Гистограмма

fig = px.histogram(df, x='species', color='species')
fig.show()

In [ ]:
#@title Species и Sex

fig = px.histogram(df.dropna(), x='species', color='sex', barmode='group')
fig.show()

> **Вывод:** пол почти никак не зависит от вида

### Island - категориальный признак (остров, где живет пингвин)

In [ ]:
#@title Информация

df['island'].describe()

count        344
unique         3
top       Biscoe
freq         168
Name: island, dtype: object

In [ ]:
#@title Значения

df['island'].value_counts()

Biscoe       168
Dream        124
Torgersen     52
Name: island, dtype: int64

In [ ]:
#@title Гистограмма

fig = px.histogram(df, x='island', color='island')
fig.show()

In [ ]:
#@title Island и Sex

fig = px.histogram(df.dropna(), x='island', color='sex', barmode='group')
fig.show()

> **Вывод:** пол почти никак не зависит от острова, нак котором живет пингвин

In [ ]:
#@title Species и Island

fig = px.histogram(df.dropna(), x='species', color='island', barmode='group')
fig.show()

### Bill length - числовой признак (длина клюва)

In [ ]:
#@title Информация

df['bill_length_mm'].describe()

count    342.000000
mean      43.921930
std        5.459584
min       32.100000
25%       39.225000
50%       44.450000
75%       48.500000
max       59.600000
Name: bill_length_mm, dtype: float64

In [ ]:
#@title Гистограмма

fig = px.histogram(df.dropna(), x='bill_length_mm', color='sex', marginal='box', barmode='overlay')
fig.show()

In [ ]:
#@title Bill length и Species, Island, Sex

fig = px.histogram(df, x='bill_length_mm', color='sex', barmode='overlay', facet_col='species', facet_row='island')
fig.show()

> **Вывод:** нелинейная зависимость относительно пола и длины клюва, НО линейная по разделению на виды

### Bill depth - числовой признак (глубина клюва)

In [ ]:
#@title Контрольные значения

df['bill_depth_mm'].describe()

count    342.000000
mean      17.151170
std        1.974793
min       13.100000
25%       15.600000
50%       17.300000
75%       18.700000
max       21.500000
Name: bill_depth_mm, dtype: float64

In [ ]:
#@title Гистограмма

fig = px.histogram(df.dropna(), x='bill_depth_mm', color='sex', marginal='box', barmode='overlay')
fig.show()

In [ ]:
#@title Bill depth и Species, Island, Sex

fig = px.histogram(df, x='bill_depth_mm', color='sex', barmode='overlay', facet_col='species', facet_row='island')
fig.show()

> **Вывод:** нелинейная зависимость относительно пола и глубины клюва, НО линейная по разделению на виды

In [ ]:
#@title Bill length, Bill depth и Species, Island, Sex

fig = px.scatter(df, x='bill_length_mm', y='bill_depth_mm', color='sex', facet_col='species', facet_row='island')
fig.show()

### Flipper length - числовой признак (длина крыла)

In [ ]:
#@title Информация

df['flipper_length_mm'].describe()

count    342.000000
mean     200.915205
std       14.061714
min      172.000000
25%      190.000000
50%      197.000000
75%      213.000000
max      231.000000
Name: flipper_length_mm, dtype: float64

In [ ]:
#@title Гистограмма

fig = px.histogram(df.dropna(), x='flipper_length_mm',  color='sex', marginal='box', barmode='overlay')
fig.show()

In [ ]:
#@title flipper_length_mm и Species, Island, Sex

fig = px.histogram(df, x='flipper_length_mm', color='sex', barmode='overlay', facet_col='species', facet_row='island')
fig.show()

In [ ]:
#@title Bill length, Bill depth, Flipper length и Species, Island, Sex

fig = px.scatter_3d(df[df['species'] == 'Gentoo'], x='bill_length_mm', y='bill_depth_mm', z='flipper_length_mm', color='sex')
fig.show()

> **Вывод:** нелинейная зависимость относительно пола и длины крыла, НО линейная по разделению на виды

### Body mass - числовой признак (масса тела)

In [ ]:
#@title Информация

df['body_mass_g'].describe()

count     342.000000
mean     4201.754386
std       801.954536
min      2700.000000
25%      3550.000000
50%      4050.000000
75%      4750.000000
max      6300.000000
Name: body_mass_g, dtype: float64

In [ ]:
#@title Гистограмма

fig = px.histogram(df.dropna(), x='body_mass_g', color='sex', marginal='box', barmode='overlay')
fig.show()

In [ ]:
#@title body_mass_g и Species, Island, Sex

fig = px.histogram(df, x='body_mass_g', color='sex', barmode='overlay', facet_col='species', facet_row='island')
fig.show()

In [ ]:
#@title Bill length, Bill depth, Flipper length и Species, Island, Sex

fig = px.scatter_3d(df[df['species'] == 'Gentoo'], x='bill_length_mm', y='bill_depth_mm', z='body_mass_g', color='sex')
fig.show()

> **Вывод:** нелинейная зависимость относительно пола и массы тела, НО линейная по разделению на виды

### Sex - бинарный признак (пол)

In [ ]:
#@title Информация

df['sex'].describe()

count      333
unique       2
top       male
freq       168
Name: sex, dtype: object

In [ ]:
#@title Значения

df['sex'].value_counts()

male      168
female    165
Name: sex, dtype: int64

In [ ]:
#@title Гистограмма

fig = px.histogram(df[df['sex'].notna()], x='sex', color='sex')
fig.show()

## Пропуски (удаление)

In [ ]:
#@title Все строки с пропусками

df[df.isna().any(axis=1)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN
10,Adelie,Torgersen,37.8,17.1,186.0,3300.0,NaN
11,Adelie,Torgersen,37.8,17.3,180.0,3700.0,NaN
47,Adelie,Dream,37.5,18.9,179.0,2975.0,NaN
178,Gentoo,Biscoe,44.5,14.3,216.0,4100.0,NaN
218,Gentoo,Biscoe,46.2,14.4,214.0,4650.0,NaN
256,Gentoo,Biscoe,47.3,13.8,216.0,4725.0,NaN
268,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,NaN


Две строки датасета имеют 5 пропусков (индексы: 3 и 271). Лучше всего из датасета их удалить

In [ ]:
#@title Удаляем строки с большим количеством пропусков

df = df.drop(index=[3, 271])
df[df.isna().any(axis=1)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN
10,Adelie,Torgersen,37.8,17.1,186.0,3300.0,NaN
11,Adelie,Torgersen,37.8,17.3,180.0,3700.0,NaN
47,Adelie,Dream,37.5,18.9,179.0,2975.0,NaN
178,Gentoo,Biscoe,44.5,14.3,216.0,4100.0,NaN
218,Gentoo,Biscoe,46.2,14.4,214.0,4650.0,NaN
256,Gentoo,Biscoe,47.3,13.8,216.0,4725.0,NaN
268,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,NaN


Как один из вариантов заполнения пропусков - заменить NaN на моду (male). Придумаем другой способ...

## Выбросы

In [ ]:
#@title Выброс 1
df[(df['sex'] == 'female') & (df['bill_length_mm'] > 51)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
293,Chinstrap,Dream,58.0,17.8,181.0,3700.0,female


In [ ]:
#@title Выброс 2
df[(df['sex'] == 'female') & (df['bill_depth_mm'] > 20)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
114,Adelie,Biscoe,39.6,20.7,191.0,3900.0,female


In [ ]:
#@title Избавляемся от выбросов
df = df[(df['sex'] != 'female') | ((df['bill_length_mm'] <= 51) & (df['bill_depth_mm'] <= 20))]
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male
...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male


## Преобразование признаков



In [ ]:
#@title Делим выборку на нецелевые признаки и целевой признак

train, test = df.dropna(), df[df.isna().any(axis=1)]

X_train, y_train = train.drop(columns=['sex']), np.array(train['sex'])
X_test, y_test = test.drop(columns=['sex']), np.array([])

In [ ]:
#@title Преобразование категориальных признаков

categorical_features = list(X_train.dtypes[X_train.dtypes == "object"].index)
categorical_transformer = Pipeline(steps=[("encoder", OneHotEncoder(sparse_output=False))])

In [ ]:
#@title Преобразование числовых признаков

numeric_features = list(X_train.dtypes[(X_train.dtypes == int) | (X_train.dtypes == float)].index)
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

In [ ]:
#@title Объединяем все получившиеся преобразования

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features)])

preprocessor

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['bill_length_mm', 'bill_depth_mm',
                                  'flipper_length_mm', 'body_mass_g']),
                                ('categorical',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['species', 'island'])])

In [ ]:
#@title Понижение размерности при помощи NeighborhoodComponentsAnalysis

pipe = Pipeline(steps=[('preprocessing', preprocessor),
                       ('nca', NeighborhoodComponentsAnalysis(n_components=2, random_state=42))]).fit(X_train, y_train)
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['species', 'island'])])),
                ('nca',
                 NeighborhoodComponentsAnalysis(n_components=2,
                                                random_state=42))])

In [ ]:
#@title Визуализация полученных признаков

X_new_train = pipe.transform(X_train)
X_new_test = pipe.transform(X_test)

fig = px.scatter(x=X_new_train[:,0], y=X_new_train[:,1], color=y_train)

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    legend_title='sex'
)

fig.show()

## Собственная реализация kNN

Введем следующие обозначения:

*  $d$ - число признаков
*  $u$ - объект тестовой выборки
*  $x_i = (x_{i1}, x_{i2}, \dots, x_{id})$ - объект обучающей выборки
*  $\mathbb{X} = \{ \; x = (x_1, x_2, \dots, x_d) \; \}$ - пространство объектов
*  $X_{i=1}^\ell = \{ \; x_i = (x_{i1}, x_{i2}, \dots, x_{id}) \; \}$ - обучающая выборка
*  $\rho : \mathbb{X}^2 \to [0, +\infty)$ - метрика

Алгоритм kNN:

1.   Подсчитываем значения $\rho(u, x_i)$, $\; i = \overline{1, \ell}$
2.   Сортируем объекты обучающей выборки в порядке **не убывания** полученных расстояний:

$$x_{[1]}, x_{[2]}, \dots, x_{[\ell]} : \rho(u, x_{[1]}) \leq \rho(u, x_{[2]}) \leq \dots \leq \rho(u, x_{[\ell]}) $$

3.   Оставляем в рассмотрении только **$k$ ближайших соседа**:

$$x_{[1]}, x_{[2]}, \dots, x_{[k]}$$

4.   Пусть $y_{[i]}$ - значение целевого категориального признака объекта $x_{[i]}$ отсортированной обучающей выборки. Тогда $a(u)$ (предсказание алгоритма) - мода среди значений $y_{[1]}$, $y_{[2]}$, $\dots$, $y_{[k]}$ **(в случае, если все веса равны единице)**:

$$a(u) = \arg \max_{y \in \mathbb{Y}} \sum_{i=1}^k [y_{[i]} = y] \cdot w(x_{[i]}) $$

In [ ]:
#@title Данные

X = np.array([[1], [2], [3], [4], [5], [6]])
y = np.array([0, 0, 0, 1, 1, 1])

u = np.array([3.5])

In [ ]:
#@title Вычисляем расстояния

dist = [distance.cityblock(u, x) for x in X]
print(*zip(X, dist))

(array([1]), 2.5) (array([2]), 1.5) (array([3]), 0.5) (array([4]), 0.5) (array([5]), 1.5) (array([6]), 2.5)


In [ ]:
#@title Позиции лучших соседей
pos = np.argsort(dist)[:3]
print(pos)

[2 3 1]


In [ ]:
#@title Расстояния до лучших соседей и их целевые признаки
dist, y_neighbors = np.array(dist)[pos], y[pos]
print(dist)
print(y_neighbors)

[0.5 0.5 1.5]
[0 1 0]


In [ ]:
#@title Вычисляем важности классов

# важность класса: вердикт алгоритма - наиболее важный класс
importance = dict.fromkeys([0, 1], 0)

# вычисляем важности классов
for i in range(3):
    importance[y_neighbors[i]] += 1 / dist[i]

print(importance)

{0: 2.6666666666666665, 1: 2.0}


In [ ]:
#@title Вердикт алгоритма
max(importance, key=importance.get)

0

In [ ]:
#@title Реализация класса

"""
Метод k ближайших соседей

Этапы:
------

1. Создание объекта класса
2. Вызов метода fit
3. Вызов метода predict
"""

class KNearestNeighbors:
    def __metric(self, u, x):
        """
        Метрика (на выбор 5 метрик)

        :param numpy.ndarray u: Объект тестовой выборки
        :param numpy.ndarray u: Объект обучающей выборки

        :return float res: Расстояние между объектами
        """

        if self._metric == 'manhattan':
            return distance.cityblock(u, x)
        elif self._metric == 'euclidean':
            return distance.euclidean(u, x)
        elif self._metric == 'chebyshev':
            return distance.chebyshev(u, x)
        elif self._metric == 'canberra':
            return distance.canberra(u, x)
        elif self._metric == 'cosine':
            return distance.cosine(u, x)

    def __weight(self, i, dist):
        """
        Вес, значимость соседа

        :param int i: Порядковый номер соседа
        :param float dist: Расстояние до соседа

        :return float res: Вес, значимость соседа
        """

        if self._weights == 'uniform':
            return 1
        elif self._weights == 'serial':
            return (self._n_neighbors - i) / self._n_neighbors
        elif self._weights == 'geometric':
            return self._q ** i
        elif self._weights == 'distance':
            return 1 / dist

    def __init__ (self, n_neighbors, weights, metric, q=None):
        """
        Конструктор

        :param int n_neighbors: Число соседей
        :param str weights: Веса соседей, их значимость
        :param str metric: Используемая метрика
        :param float q: Основание бесконечно убывающей геом. прогрессии весов соседей
        """

        self._n_neighbors = n_neighbors
        self._weights = weights
        self._metric = metric
        self._q = q

    def fit(self, X, y):
        """
        Обучение kNN. Так такового обучения нет,
        сохраняем данные об обучающей выборке

        :param pandas.core.frame.DataFrame X: Объекты со своими значениями нецелевых признаков
        :param numpy.ndarray y: Значения целевого признака объектов обучающей выборки
        """

        self._X_train = X
        self._y_train = y
        self._classes = list(set(y))

    def predict(self, X):
        """
        Предсказание значений kNN

        :param pandas.core.frame.DataFrame X: Объекты, для которых предсказываются значения

        :return numpy.ndarray y: Предсказанные алгоритмом значения
        """

        if isinstance(X, pd.core.frame.DataFrame):
            X = X.to_numpy()

        y = []

        for u in X:
            # вычисляем расстояния
            dist = [self.__metric(u, x) for x in self._X_train]

            # сортируем, получаем новый порядок, фиксируем лучших соседей
            pos = np.argsort(dist)[:self._n_neighbors]

            # расстояния до соседей, целевые признаки соседей
            dist, y_neighbors = np.array(dist)[pos], self._y_train[pos]

            # важность класса: вердикт алгоритма - наиболее важный класс
            importance = dict.fromkeys(self._classes, 0)

            # вычисляем важности классов
            for i in range(self._n_neighbors):
                importance[y_neighbors[i]] += self.__weight(i, dist[i])

            # добавляем ключ с наибольшим значением
            y.append(max(importance, key=importance.get))

        return np.array(y, dtype=object)

### manhattan

In [ ]:
#@title Обучающая выборка

clf = KNearestNeighbors(1, 'uniform', 'manhattan')
clf.fit(X_new_train, y_train)

In [ ]:
#@title Выборка для визуализации

x = np.linspace(-300, 400, 176)
y = np.linspace(-300, 400, 176)
z = np.zeros((len(y), len(x)), dtype=object)

for i in range(len(y)):
    for j in range(len(x)):
        z[i][j] = clf.predict(np.array([[x[j], y[i]]]))[0]

print(z)

[['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ...
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']]


In [ ]:
#@title Визуализация решения

fig = go.Figure()

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

# решения алгоритма
fig.add_trace(go.Contour(name='kNN', x=x, y=y, z=np.array(list(map(lambda x: list(map(lambda y: 0 if y == 'male' else 1, x)), z))),
                         contours=dict(start=0, end=1, size=2), colorscale=['#00AAFF', '#FFAAAA'], line_width=0, opacity=0.75,
                         colorbar=dict(title='sex')))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='kNN (n_neighbors=1, weights=uniform, metric=manhattan)'
)

fig.show()

### cosine

In [ ]:
#@title Обучающая выборка

clf = KNearestNeighbors(1, 'uniform', 'cosine')
clf.fit(X_new_train, y_train)

In [ ]:
#@title Выборка для визуализации

x = np.linspace(-300, 400, 176)
y = np.linspace(-300, 400, 176)
z = np.zeros((len(y), len(x)), dtype=object)

for i in range(len(y)):
    for j in range(len(x)):
        z[i][j] = clf.predict(np.array([[x[j], y[i]]]))[0]

print(z)

/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:622: RuntimeWarning:

invalid value encountered in double_scalars



[['female' 'female' 'female' ... 'female' 'female' 'female']
 ['female' 'female' 'female' ... 'female' 'female' 'female']
 ['female' 'female' 'female' ... 'female' 'female' 'female']
 ...
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']]


In [ ]:
#@title Визуализация решения

fig = go.Figure()

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

# решения алгоритма
fig.add_trace(go.Contour(name='kNN', x=x, y=y, z=np.array(list(map(lambda x: list(map(lambda y: 0 if y == 'male' else 1, x)), z))),
                         contours=dict(start=0, end=1, size=2), colorscale=['#00AAFF', '#FFAAAA'], line_width=0, opacity=0.75,
                         colorbar=dict(title='sex')))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='kNN (n_neighbors=1, weights=uniform, metric=cosine)'
)

fig.show()

### canberra

In [ ]:
#@title Обучающая выборка

clf = KNearestNeighbors(1, 'uniform', 'canberra')
clf.fit(X_new_train, y_train)

In [ ]:
#@title Выборка для визуализации

x = np.linspace(-300, 400, 176)
y = np.linspace(-300, 400, 176)
z = np.zeros((len(y), len(x)), dtype=object)

for i in range(len(y)):
    for j in range(len(x)):
        z[i][j] = clf.predict(np.array([[x[j], y[i]]]))[0]

print(z)

[['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ...
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']]


In [ ]:
#@title Визуализация решения

fig = go.Figure()

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

# решения алгоритма
fig.add_trace(go.Contour(name='kNN', x=x, y=y, z=np.array(list(map(lambda x: list(map(lambda y: 0 if y == 'male' else 1, x)), z))),
                         contours=dict(start=0, end=1, size=2), colorscale=['#00AAFF', '#FFAAAA'], line_width=0, opacity=0.75,
                         colorbar=dict(title='sex')))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='kNN (n_neighbors=1, weights=uniform, metric=canberra)'
)

fig.show()

### euclidean

In [ ]:
#@title Обучающая выборка

clf = KNearestNeighbors(1, 'uniform', 'euclidean')
clf.fit(X_new_train, y_train)

In [ ]:
#@title Выборка для визуализации

x = np.linspace(-300, 400, 176)
y = np.linspace(-300, 400, 176)
z = np.zeros((len(y), len(x)), dtype=object)

for i in range(len(y)):
    for j in range(len(x)):
        z[i][j] = clf.predict(np.array([[x[j], y[i]]]))[0]

print(z)

[['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ...
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']]


In [ ]:
#@title Визуализация решения

fig = go.Figure()

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

# решения алгоритма
fig.add_trace(go.Contour(name='kNN', x=x, y=y, z=np.array(list(map(lambda x: list(map(lambda y: 0 if y == 'male' else 1, x)), z))),
                         contours=dict(start=0, end=1, size=2), colorscale=['#00AAFF', '#FFAAAA'], line_width=0, opacity=0.75,
                         colorbar=dict(title='sex')))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='kNN (n_neighbors=1, weights=uniform, metric=euclidean)'
)

fig.show()

### chebyshev

In [ ]:
#@title Обучающая выборка

clf = KNearestNeighbors(1, 'uniform', 'chebyshev')
clf.fit(X_new_train, y_train)

In [ ]:
#@title Выборка для визуализации

x = np.linspace(-300, 400, 176)
y = np.linspace(-300, 400, 176)
z = np.zeros((len(y), len(x)), dtype=object)

for i in range(len(y)):
    for j in range(len(x)):
        z[i][j] = clf.predict(np.array([[x[j], y[i]]]))[0]

print(z)

[['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ...
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']]


In [ ]:
#@title Визуализация решения

fig = go.Figure()

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

# решения алгоритма
fig.add_trace(go.Contour(name='kNN', x=x, y=y, z=np.array(list(map(lambda x: list(map(lambda y: 0 if y == 'male' else 1, x)), z))),
                         contours=dict(start=0, end=1, size=2), colorscale=['#00AAFF', '#FFAAAA'], line_width=0, opacity=0.75,
                         colorbar=dict(title='sex')))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='kNN (n_neighbors=1, weights=uniform, metric=chebyshev)'
)

fig.show()

### euclidean + 3 соседа + distance weights

In [ ]:
#@title Обучающая выборка

clf = KNearestNeighbors(3, 'distance', 'euclidean')
clf.fit(X_new_train, y_train)

In [ ]:
#@title Выборка для визуализации

x = np.linspace(-300, 400, 176)
y = np.linspace(-300, 400, 176)
z = np.zeros((len(y), len(x)), dtype=object)

for i in range(len(y)):
    for j in range(len(x)):
        z[i][j] = clf.predict(np.array([[x[j], y[i]]]))[0]

print(z)

[['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ['female' 'female' 'female' ... 'male' 'male' 'male']
 ...
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']
 ['male' 'male' 'male' ... 'male' 'male' 'male']]


In [ ]:
#@title Визуализация решения

fig = go.Figure()

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

# решения алгоритма
fig.add_trace(go.Contour(name='kNN', x=x, y=y, z=np.array(list(map(lambda x: list(map(lambda y: 0 if y == 'male' else 1, x)), z))),
                         contours=dict(start=0, end=1, size=2), colorscale=['#00AAFF', '#FFAAAA'], line_width=0, opacity=0.75,
                         colorbar=dict(title='sex')))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='kNN (n_neighbors=3, weights=distance, metric=euclidean)'
)

fig.show()

### Собственная реализация kNN (Диаграммы Вороного)

In [ ]:
#@title Вспомогательный метод для визуализации

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

In [ ]:
#@title Диаграмма Вороного

# сама диаграмма
vor = Voronoi(X_new_train)
regions, vertices = voronoi_finite_polygons_2d(vor)

# каждому региону свой объект => свой класс
point_region = vor.point_region - 1
region_color = [None for i in range(len(point_region))]

for i in range(len(point_region)):
    region_color[point_region[i]] = y_train[point_region[i]]

# делим на мужских и женских особей
X_male = X_new_train[np.argwhere(y_train == 'male').flatten()]
X_female = X_new_train[np.argwhere(y_train == 'female').flatten()]

fig = go.Figure()

for i in range(len(regions)):
    # вершины Вороного
    points = np.append(vertices[regions[i]], [vertices[regions[i][0]]], axis=0)

    # цвет ячейки Вороного совпадает с цветом класса объекта внутри ячейки
    color = '#00AAFF' if region_color[i] == 'male' else '#FFAAAA'

    # отрисовываем ячейку
    fig.add_trace(go.Scatter(name='region ' + str(i), x=points[:,0], y=points[:,1], showlegend=False,
                             line=dict(color='black', width=1),
                             marker=dict(size=1), fill='toself', fillcolor=color, opacity=0.5))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='male', x=X_male[:,0], y=X_male[:,1],
                         mode='markers', marker_color='#00AAFF', showlegend=False,
                         marker_line_width=1, marker_size=7))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='female', x=X_female[:,0], y=X_female[:,1],
                         mode='markers', marker_color='#FFAAAA', showlegend=False,
                         marker_line_width=1, marker_size=7))

# добавляем соответствующие точки
fig.add_trace(go.Scatter(name='test', x=X_new_test[:,0], y=X_new_test[:,1],
                         mode='markers', marker_color='#DC143C', showlegend=False,
                         marker_line_width=1, marker_size=7))

fig.update_xaxes(range=[-300, 400])
fig.update_yaxes(range=[-300, 400])

fig.update_layout(
    xaxis_title='NCA1',
    yaxis_title='NCA2',
    title='Voronoi Diagram'
)

fig.show()

In [ ]:
#@title Предсказываем значения за $O(\log n)$

lines_x = np.sort(vertices[:,0])
lines_y = vertices[:,1]

for i in range(len(X_new_test)):
    x, y = X_new_test[i]

    # бин поиск по x
    l, r = 0, len(lines_x)

    # гарантированно x >= lines_x[0]
    while r - l > 1:
        mid = (l + r) // 2

        if x >= lines_x[mid]:
            l = mid
        else:
            r = mid

    print(lines_x[l], x, lines_x[l + 1])

-133.05404033276116 -133.0180042691708 -132.79415652766488
40.299455085029265 41.22911246566861 41.64009041931955
-140.37815590693478 -139.27398827016668 -137.24379185242285
-102.69019720868584 -101.8254027930975 -101.40103121581136
-162.45023796572266 -162.11116623154237 -161.904467769208
4.139769614619105 4.332062517328122 4.662861869956849
69.70527823356302 70.8146840594144 71.01757424250529
77.57754850578841 77.93575172784715 78.14386692956013
100.7888113132276 101.64134199810425 103.23880598601563
